Given a m x n binary matrix mat. In one step, you can choose one cell and flip it and all the four neighbours of it if they exist (Flip is changing 1 to 0 and 0 to 1). A pair of cells are called neighboors if they share one edge.

Return the minimum number of steps required to convert mat to a zero matrix or -1 if you cannot.

Binary matrix is a matrix with all cells equal to 0 or 1 only.

Zero matrix is a matrix with all cells equal to 0.

 

Example 1:

```
Input: mat = [[0,0],[0,1]]
Output: 3
Explanation: One possible solution is to flip (1, 0) then (0, 1) and finally (1, 1) as shown.
```
Example 2:
```
Input: mat = [[0]]
Output: 0
Explanation: Given matrix is a zero matrix. We don't need to change it.
``` 

Example 3:
```
Input: mat = [[1,1,1],[1,0,1],[0,0,0]]
Output: 6
```
Example 4:
```
Input: mat = [[1,0,0],[1,0,0]]
Output: -1
Explanation: Given matrix can't be a zero matrix
``` 

Constraints:

- m == mat.length
- n == mat[0].length
- 1 <= m <= 3
- 1 <= n <= 3
- mat[i][j] is 0 or 1.

In [ ]:
# BFS + bitmap + memorization + brutal
import queue
class Solution:
    def encode(self, mat, m, n):
        x = 0
        for i in range(m):
            for j in range(n):
                x = 2*x + mat[i][j]
        return x
    def decode(self, x, m, n):
        mat = [[0] * n for _ in range(m)]
        for i in range(m-1, -1, -1):
            for j in range(n-1,-1,-1):
                mat[i][j] = x & 1
                x >>= 1
        return mat
    def convert(self, mat, i, j, m, n):        
        for m0, n0 in [(0,1), (1,0), (-1,0), (0,-1), (0,0)]:
            if 0 <= m0+i < m and 0 <= n0+j < n:
                mat[m0+i][n0+j] ^= 1
        
        
    def minFlips(self, mat: List[List[int]]) -> int:
        m, n = len(mat), len(mat[0])
        x_start = self.encode(mat, m, n)
        step = 0
        if x_start == 0:
            return step
        visited = {x_start}
        q = queue.Queue()
        q.put_nowait(x_start)
        
        while not q.empty():
            step += 1
            k = q.qsize()
            #print(k)
            for _ in range(k):
                status = self.decode(q.get_nowait(), m, n)
                for i in range(m):
                    for j in range(n):
                        self.convert(status, i, j, m, n)
                        x = self.encode(status, m, n)
                        
                        if x == 0:
                            return step
                        if x not in visited:
                            visited.add(x)
                            q.put_nowait(x)
                        self.convert(status, i, j, m, n)
        return -1
                            
                            

In [ ]:
# DFS 方法， 这样的好处是不用记录是否访问过，时间复杂度同上O(2^(MN) *(MN))
import math
class Solution:
    def __init__(self):
        self.ans = 10**9
    def convert(self, mat, m, n, i, j):
        for m0, n0 in [(0,1),(0,-1),(0,0),(1,0),(-1,0)]:
            new_m, new_n = i+m0, j+n0
            if 0 <= new_m < m and 0 <= new_n < n:
                mat[new_m][new_n] ^= 1
    def dfs(self, mat, m, n, pos, count):
        if pos == m*n:
            if all(mat[i][j] == 0 for i in range(m) for j in range(n)):
                self.ans = min(self.ans, count)
            return
        x, y = divmod(pos, n)
        self.dfs(mat, m, n, pos+1, count)
        self.convert(mat, m, n, x, y)
        self.dfs(mat, m, n, pos+1, count+1)
        self.convert(mat, m, n, x, y)
        
    def minFlips(self, mat:List[List[int]]) -> int:
        m, n = len(mat), len(mat[0])
        self.dfs(mat, m, n, 0, 0)
        return self.ans if self.ans != 10**9 else -1

In [ ]:
# DFS算法+可行性判定，即如果该点上面是1，那么这个点必须翻转，不用考虑不翻转的情况，时间复杂度降为了O(2^N * M*N)
import math
class Solution:
    def __init__(self):
        self.ans = math.inf
    def convert(self, mat, m, n, i, j):
        for di, dj in [(0, -1), (0, 1),(0, 0), (-1, 0), (1, 0)]:
            m0, n0 = i + di, j + dj
            if 0 <= m0 < m and 0 <= n0 < n:
                mat[m0][n0] ^= 1
    def dfs(self, mat, m, n, pos, step):
        if pos == m*n:
            if all(1 not in mat[i] for i in range(m)):
                self.ans = min(self.ans, step)
            return
        if pos < n:
            self.dfs(mat, m, n, pos + 1, step)
            self.convert(mat, m, n, 0, pos)
            self.dfs(mat, m, n, pos + 1, step + 1)
            self.convert(mat, m, n, 0, pos)
        else:
            x, y = divmod(pos, n)
            if mat[x-1][y] == 1:
                self.convert(mat, m, n, x, y)
                self.dfs(mat, m, n, pos + 1, step + 1)
                self.convert(mat, m, n, x, y)
            else:
                self.dfs(mat, m, n, pos + 1, step)
    def minFlips(self, mat: List[List[int]]) -> int:
        m, n = len(mat), len(mat[0])
        self.dfs(mat, m, n, 0, 0)
        return self.ans if self.ans != math.inf else -1

In [ ]:
# 是否还能更优化时间复杂度呢？应该是可以的，我们可以在判断是否全为0矩阵时，只用最后一行判断即可，时间复杂度O(2^N *N)
import math
class Solution:
    def __init__(self):
        self.ans = math.inf
    def convert(self, mat, m, n, i, j):
        for di, dj in [(0, -1), (0, 1),(0, 0), (-1, 0), (1, 0)]:
            m0, n0 = i + di, j + dj
            if 0 <= m0 < m and 0 <= n0 < n:
                mat[m0][n0] ^= 1
    def dfs(self, mat, m, n, pos, step):
        if pos == m*n:
            if 1 not in mat[-1]:
                self.ans = min(self.ans, step)
            return
        if pos < n:
            self.dfs(mat, m, n, pos + 1, step)
            self.convert(mat, m, n, 0, pos)
            self.dfs(mat, m, n, pos + 1, step + 1)
            self.convert(mat, m, n, 0, pos)
        else:
            x, y = divmod(pos, n)
            if mat[x-1][y] == 1:
                self.convert(mat, m, n, x, y)
                self.dfs(mat, m, n, pos + 1, step + 1)
                self.convert(mat, m, n, x, y)
            else:
                self.dfs(mat, m, n, pos + 1, step)
    def minFlips(self, mat: List[List[int]]) -> int:
        m, n = len(mat), len(mat[0])
        self.dfs(mat, m, n, 0, 0)
        return self.ans if self.ans != math.inf else -1
    

## 如果这道题要求返回需要翻转的位置呢？
- 如果不考虑爆仓的可能行，可以每次递归的时候记录DFS之前翻转的位置
- 如果可以牺牲一点点时间，可以只记录第一行的位置，而后根据可行性判断补足整个矩阵中的位置

In [25]:
import math
class Solution:
    def __init__(self):
        self.ans = math.inf
        self.result = []
    def convert(self, mat, m, n, i, j):
        for di, dj in [(0,1),(0,0), (0, -1),(1, 0), (-1, 0)]:
            m0, n0 = i + di, j + dj
            if 0 <= m0 < m and 0 <= n0 < n:
                mat[m0][n0] ^= 1
    def dfs(self, mat, m, n, pos, count, flip):
        #print(flip)
        if pos == m*n:
            if 1 not in mat[-1]:
                if self.ans >= count:
                    self.ans = count
                    self.result = flip
            return
        if pos < n:
            self.dfs(mat, m, n, pos + 1, count, flip)
            self.convert(mat, m, n, 0, pos)
            #flip.append(pos)
            self.dfs(mat, m, n, pos + 1, count + 1, flip + [pos])
            #flip.pop()
            self.convert(mat, m, n, 0, pos)
        else:
            x, y = divmod(pos, n)
            if mat[x-1][y] == 1:
                self.convert(mat, m, n, x, y)
                self.dfs(mat, m, n, pos + 1, count + 1, flip)
                self.convert(mat, m, n, x, y)
            else:
                self.dfs(mat, m, n, pos + 1, count, flip)
    def minFlips(self, mat) -> int:
        m, n = len(mat), len(mat[0])
        self.dfs(mat, m, n, 0, 0, [])
        return self.result

In [26]:
if __name__ == '__main__':
    mat = [[0, 0], [0, 1]]
    solution = Solution()
    print(solution.minFlips(mat))

[1]


In [8]:
[] + [1]

[1]